In [1]:
import torch

In [2]:
import os
import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity

import anndata as ad
from scarches.dataset.trvae.data_handling import remove_sparsity
from scarches.models.scpoli import scPoli

import warnings
warnings.filterwarnings('ignore')
# %load_ext autoreload
# %autoreload 2

 captum (see https://github.com/pytorch/captum).
INFO:pytorch_lightning.utilities.seed:Global seed set to 0
/gpfs/space/home/dzvenymy/.conda/envs/scarches_p_3_10/lib/python3.10/site-packages/pytorch_lightning-1.7.7-py3.10.egg/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/gpfs/space/home/dzvenymy/.conda/envs/scarches_p_3_10/lib/python3.10/site-packages/pytorch_lightning-1.7.7-py3.10.egg/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


# Useful functions

In [ ]:
def sample_adata_for_viz(adata, frac=0.3, rs=3):
    indx = adata.obs.sample(frac=frac, random_state=rs, ignore_index=True)
    obs = adata.obs.iloc[list(indx.index)]
    x = adata.X[list(indx.index)]
    adata_frac = sc.AnnData(X=x, var=adata.var, obs=obs)
    return adata_frac

# OneK1K data

In [3]:
adata = sc.read('/gpfs/space/home/dzvenymy/scarches/data/OneK1K/local.h5ad')

## Data exploration

In [4]:
adata.obs['barcode_not_unique'] = adata.obs.index
adata.obs['barcode_not_unique'] = adata.obs['barcode_not_unique'].apply(lambda x: x.split('-')[0])


In [6]:
len(adata.obs['barcode_not_unique'].unique())ad.concat(adatas, label="dataset")

603228

In [14]:
len(adata.obs)

1248980

In [16]:
adata.obs.head(5)

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,donor_id,pool_number,predicted.celltype.l2,predicted.celltype.l2.score,age,organism_ontology_term_id,...,is_primary_data,suspension_type,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage
barcode,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGAATGTTG-1,onek1k,4921.0,1256,1.341191,691_692,1,CD4 TCM,0.605378,66,NCBITaxon:9606,...,True,cell,"central memory CD4-positive, alpha-beta T cell",10x 3' v2,normal,Homo sapiens,male,blood,European,66-year-old human stage
AAACCTGAGAGAACAG-1,onek1k,2523.0,941,3.606817,693_694,1,NK,1.000000,88,NCBITaxon:9606,...,True,cell,natural killer cell,10x 3' v2,normal,Homo sapiens,female,blood,European,88-year-old human stage
AAACCTGAGCATGGCA-1,onek1k,2565.0,716,3.274854,688_689,1,CD4 Naive,0.557355,66,NCBITaxon:9606,...,True,cell,"naive thymus-derived CD4-positive, alpha-beta ...",10x 3' v2,normal,Homo sapiens,male,blood,European,66-year-old human stage
AAACCTGAGTATTGGA-1,onek1k,8992.0,1859,2.524466,683_684,1,CD8 TEM,0.359614,78,NCBITaxon:9606,...,True,cell,"effector memory CD8-positive, alpha-beta T cell",10x 3' v2,normal,Homo sapiens,male,blood,European,78-year-old human stage
AAACCTGAGTGTCCCG-1,onek1k,3591.0,992,4.678363,684_685,1,CD8 TEM,0.896674,75,NCBITaxon:9606,...,True,cell,"effector memory CD8-positive, alpha-beta T cell",10x 3' v2,normal,Homo sapiens,female,blood,European,75-year-old human stage


In [21]:
adata.obs.tail(5)

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,donor_id,pool_number,predicted.celltype.l2,predicted.celltype.l2.score,age,organism_ontology_term_id,...,is_primary_data,suspension_type,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage
barcode,,,,,,,,,,,,,,,,,,,,,
TTTGTCATCCGCTGTT-9,onek1k,4321.0,1073,3.031706,796_797,9,B intermediate,0.811524,64,NCBITaxon:9606,...,True,cell,transitional stage B cell,10x 3' v2,normal,Homo sapiens,male,blood,European,64-year-old human stage
TTTGTCATCCGTTGTC-9,onek1k,4890.0,1013,2.106339,800_801,9,CD4 TCM,0.922046,37,NCBITaxon:9606,...,True,cell,"central memory CD4-positive, alpha-beta T cell",10x 3' v2,normal,Homo sapiens,female,blood,European,37-year-old human stage
TTTGTCATCGCCGTGA-9,onek1k,4636.0,919,2.243313,821_822,9,CD4 Naive,0.763648,90,NCBITaxon:9606,...,True,cell,"naive thymus-derived CD4-positive, alpha-beta ...",10x 3' v2,normal,Homo sapiens,male,blood,European,90-year-old human stage
TTTGTCATCGCGGATC-9,onek1k,4536.0,1168,3.306878,840_841,9,CD4 TCM,0.865812,75,NCBITaxon:9606,...,True,cell,"central memory CD4-positive, alpha-beta T cell",10x 3' v2,normal,Homo sapiens,female,blood,European,75-year-old human stage
TTTGTCATCTCGTATT-9,onek1k,4387.0,1300,2.165489,821_822,9,CD4 TEM,0.917827,90,NCBITaxon:9606,...,True,cell,"effector memory CD4-positive, alpha-beta T cell",10x 3' v2,normal,Homo sapiens,male,blood,European,90-year-old human stage


In [17]:
adata.obs.columns

Index(['orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'donor_id',
       'pool_number', 'predicted.celltype.l2', 'predicted.celltype.l2.score',
       'age', 'organism_ontology_term_id', 'tissue_ontology_term_id',
       'assay_ontology_term_id', 'disease_ontology_term_id',
       'cell_type_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id',
       'development_stage_ontology_term_id', 'sex_ontology_term_id',
       'is_primary_data', 'suspension_type', 'cell_type', 'assay', 'disease',
       'organism', 'sex', 'tissue', 'self_reported_ethnicity',
       'development_stage'],
      dtype='object')

In [18]:
adata.obs['cell_type_ontology_term_id'].unique()

['CL:0000904', 'CL:0000623', 'CL:0000895', 'CL:0000913', 'CL:0000788', ..., 'CL:2000001', 'CL:0000784', 'CL:0000625', 'CL:0000624', 'CL:0000451']
Length: 29
Categories (29, object): ['CL:0000232', 'CL:0000233', 'CL:0000451', 'CL:0000623', ..., 'CL:0002396', 'CL:0002489', 'CL:0008001', 'CL:2000001']

In [19]:
adata.obs['cell_type'].unique()

['central memory CD4-positive, alpha-beta T cell', 'natural killer cell', 'naive thymus-derived CD4-positive, alpha-beta..., 'effector memory CD8-positive, alpha-beta T cell', 'naive B cell', ..., 'peripheral blood mononuclear cell', 'plasmacytoid dendritic cell', 'CD8-positive, alpha-beta T cell', 'CD4-positive, alpha-beta T cell', 'dendritic cell']
Length: 29
Categories (29, object): ['erythrocyte', 'platelet', 'dendritic cell', 'natural killer cell', ..., 'CD14-low, CD16-positive monocyte', 'double negative thymocyte', 'hematopoietic precursor cell', 'peripheral blood mononuclear cell']

In [20]:
adata.obs['pool_number'].unique()

['1', '10', '11', '12', '13', ..., '75', '76', '77', '8', '9']
Length: 75
Categories (75, object): ['1', '2', '3', '4', ..., '74', '75', '76', '77']

In [76]:
adata.obs['pool_number'].value_counts().loc['41']

18940

## Get cell types for each pool counts

In [3]:
meta = pd.read_csv('../data/OneK1K/OneK1K_SRR_metadata.csv', sep='\t')
meta.head()

,id,accession,run_accession,experiment_accession,sample_accession,secondary_sample_accession,study_accession,secondary_study_accession,parent_study,submission_accession,...,fastq_md5,fastq_bytes,fastq_ftp,fastq_galaxy,fastq_aspera,fastq_1,fastq_2,md5_1,md5_2,single_end
0,SRX14182503_SRR18028378,SAMN25948860,SRR18028378,SRX14182503,SAMN25948860,SRS12006093,PRJNA807049,SRP359840,NaN,SRA1372903,...,6ea049161c30f920a168a4ec7231e4b4,2139539305,ftp.sra.ebi.ac.uk/vol1/fastq/SRR180/078/SRR180...,ftp.sra.ebi.ac.uk/vol1/fastq/SRR180/078/SRR180...,fasp.sra.ebi.ac.uk:/vol1/fastq/SRR180/078/SRR1...,ftp.sra.ebi.ac.uk/vol1/fastq/SRR180/078/SRR180...,NaN,6ea049161c30f920a168a4ec7231e4b4,NaN,True
1,SRX14182503_SRR18028379,SAMN25948860,SRR18028379,SRX14182503,SAMN25948860,SRS12006093,PRJNA807049,SRP359840,NaN,SRA1372903,...,cee2592736a279539a109b333b3f109b,1981957689,ftp.sra.ebi.ac.uk/vol1/fastq/SRR180/079/SRR180...,ftp.sra.ebi.ac.uk/vol1/fastq/SRR180/079/SRR180...,fasp.sra.ebi.ac.uk:/vol1/fastq/SRR180/079/SRR1...,ftp.sra.ebi.ac.uk/vol1/fastq/SRR180/079/SRR180...,NaN,cee2592736a279539a109b333b3f109b,NaN,True
2,SRX14182503_SRR18028380,SAMN25948860,SRR18028380,SRX14182503,SAMN25948860,SRS12006093,PRJNA807049,SRP359840,NaN,SRA1372903,...,ba9622076695ff8e67bbf93c31a8f8f8,1989248167,ftp.sra.ebi.ac.uk/vol1/fastq/SRR180/080/SRR180...,ftp.sra.ebi.ac.uk/vol1/fastq/SRR180/080/SRR180...,fasp.sra.ebi.ac.uk:/vol1/fastq/SRR180/080/SRR1...,ftp.sra.ebi.ac.uk/vol1/fastq/SRR180/080/SRR180...,NaN,ba9622076695ff8e67bbf93c31a8f8f8,NaN,True
3,SRX14182503_SRR18028381,SAMN25948860,SRR18028381,SRX14182503,SAMN25948860,SRS12006093,PRJNA807049,SRP359840,NaN,SRA1372903,...,73f50d2c8af5757063c967e9124addba,1970104845,ftp.sra.ebi.ac.uk/vol1/fastq/SRR180/081/SRR180...,ftp.sra.ebi.ac.uk/vol1/fastq/SRR180/081/SRR180...,fasp.sra.ebi.ac.uk:/vol1/fastq/SRR180/081/SRR1...,ftp.sra.ebi.ac.uk/vol1/fastq/SRR180/081/SRR180...,NaN,73f50d2c8af5757063c967e9124addba,NaN,True
4,SRX14182503_SRR18028382,SAMN25948860,SRR18028382,SRX14182503,SAMN25948860,SRS12006093,PRJNA807049,SRP359840,NaN,SRA1372903,...,5d42d1f4fcbb3a0c6618d1809725fd00,2099757277,ftp.sra.ebi.ac.uk/vol1/fastq/SRR180/082/SRR180...,ftp.sra.ebi.ac.uk/vol1/fastq/SRR180/082/SRR180...,fasp.sra.ebi.ac.uk:/vol1/fastq/SRR180/082/SRR1...,ftp.sra.ebi.ac.uk/vol1/fastq/SRR180/082/SRR180...,NaN,5d42d1f4fcbb3a0c6618d1809725fd00,NaN,True


In [30]:
meta.columns

Index(['id', 'accession', 'run_accession', 'experiment_accession',
       'sample_accession', 'secondary_sample_accession', 'study_accession',
       'secondary_study_accession', 'parent_study', 'submission_accession',
       'run_alias', 'experiment_alias', 'sample_alias', 'study_alias',
       'library_layout', 'library_selection', 'library_source',
       'library_strategy', 'library_name', 'instrument_model',
       'instrument_platform', 'base_count', 'read_count', 'tax_id',
       'scientific_name', 'sample_title', 'experiment_title', 'study_title',
       'description', 'sample_description', 'fastq_md5', 'fastq_bytes',
       'fastq_ftp', 'fastq_galaxy', 'fastq_aspera', 'fastq_1', 'fastq_2',
       'md5_1', 'md5_2', 'single_end'],
      dtype='object')

In [35]:
meta['sample_title']

0        PMBCs for 12 individuals in pool 1
1        PMBCs for 12 individuals in pool 1
2        PMBCs for 12 individuals in pool 1
3        PMBCs for 12 individuals in pool 1
4        PMBCs for 12 individuals in pool 1
                       ...                 
1495    PMBCs for 15 individuals in pool 65
1496    PMBCs for 15 individuals in pool 65
1497    PMBCs for 15 individuals in pool 65
1498    PMBCs for 15 individuals in pool 65
1499    PMBCs for 15 individuals in pool 65
Name: sample_title, Length: 1500, dtype: object

In [4]:
sample_names = set(meta['sample_accession'].unique())
len(sample_names)

75

In [6]:
COUNTS_ROOT = '/gpfs/space/projects/genomic_references/scRNAseq_counts/kallisto/OneK1K/'
sample_names_from_counts = set(map(lambda x: x.split('.')[0], os.listdir(COUNTS_ROOT)))
len(sample_names_from_counts)

75

In [29]:
len(sample_names & sample_names_from_counts)

75

In [5]:
sn2pool = {k: v.split(' ')[-1] for k,v in list((meta.groupby(['sample_accession', 'sample_title']).groups.keys()))}

In [6]:
len(sn2pool)

75

In [9]:
for sn in sample_names:
    adata_sample = sc.read(os.path.join(COUNTS_ROOT, sn + '.h5ad'))
    cell_type_df = pd.DataFrame(adata.obs[adata.obs['pool_number'] == sn2pool[sn]]['cell_type'])
    cell_type_df['barcode'] = cell_type_df.index.to_series()
    cell_type_df['barcode'] = cell_type_df['barcode'].apply(lambda x: x.split('-')[0])
    cell_type_df = cell_type_df.set_index('barcode')
    
    adata_sample.obs['cell_type'] = cell_type_df['cell_type'].reindex(adata_sample.obs.index)
    
    print(len(adata_sample.obs[adata_sample.obs['cell_type'].notnull()]) == len(cell_type_df))
    
    X = adata_sample.X[adata_sample.obs.index.get_indexer(adata_sample.obs[adata_sample.obs['cell_type'].notnull()].index)]
    
    adata_sample_new = ad.AnnData(X, obs=adata_sample.obs.dropna(), var=adata_sample.var)
    
    adata_sample_new.write(f'../data/OneK1K/counts/{sn}.h5ad')
   

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


## Merge all samples in one adata

In [6]:
d = {}
for sn in sample_names:
    adata_sample = sc.read(f'../data/OneK1K/counts/{sn}.h5ad')
    adata_sample.obs = adata_sample.obs.rename(index=dict(zip(adata_sample.obs.index, 
                                                              list(map(lambda x: x + '-' + sn2pool[sn], adata_sample.obs.index)))))
    d[sn] = adata_sample
adata_onek1k= ad.concat(d, label="sample")

In [9]:
del d

In [10]:
len(adata_onek1k1)

1248980

In [19]:
adata_onek1k1.obs.head(5)

,cell_type,sample,cell_type_id
barcode,,,
AAACCTGAGAAGATTC-65,"central memory CD4-positive, alpha-beta T cell",SAMN25948905,0
AAACCTGAGATACACA-65,"effector memory CD8-positive, alpha-beta T cell",SAMN25948905,1
AAACCTGAGGGAACGG-65,natural killer cell,SAMN25948905,2
AAACCTGAGGTACTCT-65,"effector memory CD8-positive, alpha-beta T cell",SAMN25948905,1
AAACCTGAGTTATCGC-65,"naive thymus-derived CD4-positive, alpha-beta ...",SAMN25948905,3


In [9]:
ct2id =  {ct:idd for idd, ct in enumerate(adata_onek1k.obs['cell_type'].unique())}
adata_onek1k.obs['cell_type_id'] = adata_onek1k.obs['cell_type'].apply(lambda x: ct2id[x])

# Perez data

In [3]:
adata = sc.read_h5ad('/gpfs/space/home/dzvenymy/scarches/data/Perez/local.h5ad', backed='r')

In [19]:
len(adata)

1263676

In [4]:
adata.obs.head(5)

,library_uuid,assay_ontology_term_id,mapped_reference_annotation,is_primary_data,cell_type_ontology_term_id,author_cell_type,cell_state,sample_uuid,tissue_ontology_term_id,development_stage_ontology_term_id,...,ct_cov,ind_cov,cell_type,assay,disease,organism,sex,tissue,self_reported_ethnicity,development_stage
index,,,,,,,,,,,,,,,,,,,,,
CAAGGCCAGTATCGAA-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-0-0-0-0-0,3332f018-936e-4c4c-9105-99d9503db5a3,EFO:0009899,GENCODE 19,True,CL:0000624,T4,na,0a1148dc-356f-4dc4-9c32-bf0823257d93,UBERON:0000178,HsapDv:0000122,...,T4_naive,HC-546,"CD4-positive, alpha-beta T cell",10x 3' v2,normal,Homo sapiens,female,blood,Asian,28-year-old human stage
CTAACTTCAATGAATG-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-0-0-0-0-0,70a004b7-4a17-4702-8910-4557aa0c4279,EFO:0009899,GENCODE 19,True,CL:0000860,cM,na,577bce84-8d37-4851-9fb4-53f9467699ba,UBERON:0000178,HsapDv:0000139,...,NaN,1132_1132,classical monocyte,10x 3' v2,systemic lupus erythematosus,Homo sapiens,female,blood,European,45-year-old human stage
AAGTCTGGTCTACCTC-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,152e2bfd-e9ea-4d70-a999-6f37fb3fb96c,EFO:0009899,GENCODE 19,True,CL:0000860,cM,na,0f3a21f5-e7b9-4139-93fb-5407dbfde4fc,UBERON:0000178,HsapDv:0000128,...,NaN,FLARE006,classical monocyte,10x 3' v2,systemic lupus erythematosus,Homo sapiens,female,blood,European,34-year-old human stage
GGCTCGATCGTTGACA-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-0-0-0-0-0-0-0-0-0-0,c2641f62-eb23-4dad-9c22-b52e72b79df2,EFO:0009899,GENCODE 19,True,CL:0000236,B,na,5e0ed28c-a75a-4ecd-a0c7-49e71264690b,UBERON:0000178,HsapDv:0000165,...,B_naive,1110_1110,B cell,10x 3' v2,systemic lupus erythematosus,Homo sapiens,female,blood,European,71-year-old human stage
ACACCGGCACACAGAG-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,222b358b-71e7-4b0f-9f9b-47b4c67aaa27,EFO:0009899,GENCODE 19,True,CL:0000624,T4,na,982d60b4-5677-4e2d-8b58-79503863710d,UBERON:0000178,HsapDv:0000122,...,NaN,1479_1479,"CD4-positive, alpha-beta T cell",10x 3' v2,systemic lupus erythematosus,Homo sapiens,female,blood,Asian,28-year-old human stage


In [5]:
adata.obs.columns

Index(['library_uuid', 'assay_ontology_term_id', 'mapped_reference_annotation',
       'is_primary_data', 'cell_type_ontology_term_id', 'author_cell_type',
       'cell_state', 'sample_uuid', 'tissue_ontology_term_id',
       'development_stage_ontology_term_id', 'disease_state',
       'suspension_enriched_cell_types', 'suspension_uuid', 'suspension_type',
       'donor_id', 'self_reported_ethnicity_ontology_term_id',
       'organism_ontology_term_id', 'disease_ontology_term_id',
       'sex_ontology_term_id', 'Processing_Cohort', 'ct_cov', 'ind_cov',
       'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue',
       'self_reported_ethnicity', 'development_stage'],
      dtype='object')

In [6]:
adata.obs['cell_type'].unique()

['CD4-positive, alpha-beta T cell', 'classical monocyte', 'B cell', 'CD8-positive, alpha-beta T cell', 'natural killer cell', ..., 'lymphocyte', 'conventional dendritic cell', 'plasmacytoid dendritic cell', 'plasmablast', 'progenitor cell']
Length: 11
Categories (11, object): ['B cell', 'lymphocyte', 'natural killer cell', 'CD4-positive, alpha-beta T cell', ..., 'non-classical monocyte', 'plasmablast', 'conventional dendritic cell', 'progenitor cell']

In [18]:
adata.obs['ind_cov'].unique()

['HC-546', '1132_1132', 'FLARE006', '1110_1110', '1479_1479', ..., 'FLARE005', '1597_1597', 'HC-508', '1493_1493', 'HC-566']
Length: 261
Categories (261, object): ['1004_1004', '1014_1014', '1015_1015', '1019_1019', ..., 'IGTB1987_IGTB1987', 'IGTB1996_IGTB1996', 'IGTB2007_IGTB2007', 'IGTB2065_IGTB2065']

## Get cell types

In [23]:
COUNTS_ROOT = '/gpfs/space/projects/genomic_references/scRNAseq_counts/kallisto/Perez_2022/'
sample_names_from_counts = set(map(lambda x: x.split('.')[0], os.listdir(COUNTS_ROOT)))
len(sample_names_from_counts)

66

In [24]:
l = {}
for sn in sample_names_from_counts:
    adata_sample = sc.read(os.path.join(COUNTS_ROOT, sn + '.h5ad'))
    l[sn] = len(adata_sample)

In [25]:
sum([v for v in l.values()])

35032220

# Train scpoli model

In [10]:
condition_key = 'sample'
cell_type_key = ['cell_type_id']

In [11]:
scpoli_model = scPoli(
    adata=adata_onek1k,
    condition_key=condition_key,
    cell_type_keys=cell_type_key,
    embedding_dim=20,
    latent_dim=30,
    hidden_layer_sizes=[128,128,128],
    dr_rate=0.1,
    use_ln=True
)

Embedding dictionary:
 	Num conditions: 75
 	Embedding dim: 20
Encoder Architecture:
	Input Layer in, out and cond: 61533 128 20
	Hidden Layer 1 in/out: 128 128
	Hidden Layer 2 in/out: 128 128
	Mean/Var Layer in/out: 128 30
Decoder Architecture:
	First Layer in, out and cond:  30 128 20
	Hidden Layer 1 in/out: 128 128
	Hidden Layer 2 in/out: 128 128
	Output Layer in/out:  128 61533 



In [ ]:
early_stopping_kwargs = {
    "early_stopping_metric": "val_prototype_loss",
    "mode": "min",
    "threshold": 0,
    "patience": 20,
    "reduce_lr": True,
    "lr_patience": 13,
    "lr_factor": 0.1,
}

scpoli_model.train(
    n_epochs=50,
    pretraining_epochs=40,
    early_stopping_kwargs=early_stopping_kwargs,
    eta=5,
    batch_size=128 # default is 128
)

 |███████████---------| 56.0%  - val_loss: 3897.4189678255 - val_trvae_loss: 3897.4189678255

In [ ]:
scpoli_model.save('../models/onek1k_only/')

# Evaluation

## Visualize Onek1k only

In [ ]:
adata_onek1k_frac = sample_adata_for_viz(adata_onek1k1, frac=0.5, rs=42)

In [ ]:
scpoli_model.model.eval()
data_latent_source = scpoli_model.get_latent(
    adata_onek1k_frac.X.toarray(), 
    adata_onek1k_frac.obs[condition_key].values,
    mean=True
)

adata_latent_source = sc.AnnData(data_latent_source)
adata_latent_source.obs = adata_onek1k_frac.obs.copy()

In [ ]:
sc.pp.neighbors(adata_latent_source, n_neighbors=15)
sc.tl.umap(adata_latent_source)

In [ ]:
sc.pl.umap(
    adata_latent_source, 
    color='sample',
    show=False, 
    frameon=False,
    palette=matplotlib.rcParams["axes.prop_cycle"]
    # legend_loc=None,
)

In [ ]:
sc.pl.umap(
    adata_latent_source, 
    color='cell_type',
    show=False, 
    frameon=False,
    palette=matplotlib.rcParams["axes.prop_cycle"]
    # legend_loc=None,
)

## Perez query mapping

In [ ]:
adata_perez_query
adata_perez_train

In [ ]:
scpoli_model = scPoli.load('../models/onek1k_only/', adata_onek1k1)

scpoli_query = scPoli.load_query_data(
    adata=adata_perez_query,
    reference_model=scpoli_model,
    labeled_indices=[],
)

In [ ]:
scpoli_query.train(
    n_epochs=50,
    pretraining_epochs=40,
    eta=5
)